### Connect to GIS

In [1]:
from arcgis import GIS
from arcgis.features import SpatialDataFrame, FeatureLayer

In [2]:
gis = GIS('https://deldev.maps.arcgis.com', 'deldev', 'P@ssw0rd@123', profile='deldev')

### Load crash data

In [3]:
import pandas as pd

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [5]:
df = pd.read_pickle('df_utah.pkl')

In [6]:
df.head()

,CRASHID,RouteName,BeginMilePost,RoadView,CrashSeverity,MANNERCOLLISIONID,ROADWAYJUNCTFEATUREID,NUMBERVEHICLESINVOLVED,FIRSTHARMFULEVENTID,ROADWAYTYPECD,ROUTEDIRECTION,EXITNUMBER,RAMPID,Latitude,Longitude,db,project,run,lat,lon,dayofyear,year,month,day,hour,dayofweek,incident
DATE,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-10-15 19:56:00,10401151,0009P,11.64,http://168.178.125.111/virtualnavigator/udot/V...,2,Front to Rear,No Special Feature/Junction,2,Other Motor Vehicle in Transport,Mainline,P,NaN,NaN,4119389.0,298229.0,2014,0009,P,37.199177,-113.273482,288,2010,10,15,19,4,1
2010-08-20 19:33:00,10394697,0009P,11.85,http://168.178.125.111/virtualnavigator/udot/V...,1,Sideswipe Same Direction,No Special Feature/Junction,2,Other Motor Vehicle in Transport,Mainline,P,NaN,NaN,4119704.0,298349.0,2014,0009,P,37.202040,-113.272216,232,2010,8,20,19,4,1
2010-03-27 21:30:00,10346333,0009P,14.00,http://168.178.125.111/virtualnavigator/udot/V...,1,Single Vehicle,No Special Feature/Junction,1,Delineator Post,Mainline,P,NaN,NaN,4122320.0,299671.0,2014,0009,P,37.225887,-113.258033,86,2010,3,27,21,5,1
2010-04-16 13:45:00,10352753,0009P,14.00,http://168.178.125.111/virtualnavigator/udot/V...,1,Single Vehicle,No Special Feature/Junction,1,Overturn/Rollover,Mainline,P,NaN,NaN,4122320.0,299671.0,2014,0009,P,37.225887,-113.258033,106,2010,4,16,13,4,1
2010-03-05 09:48:00,10357506,0009P,21.00,http://168.178.125.111/virtualnavigator/udot/V...,1,Single Vehicle,No Special Feature/Junction,1,Animal - Wild,Mainline,P,NaN,NaN,4118637.0,309403.0,2014,0009,P,37.194752,-113.147468,64,2010,3,5,9,4,1


In [7]:
df.shape

(430745, 27)

In [8]:
df[df.RAMPID.notnull()].shape

(16179, 27)

In [9]:
df['RouteName'] = df['RouteName'].apply(lambda x: x.zfill(5)) # mislabeled, zero pad so it matches LRS names
df.loc[df.RouteName == '0011P', 'RouteName'] = '089AP'        # mislabeled - QA from ArcGIS Pro inspection

In [10]:
route_names = df.RouteName.unique()
len(route_names)

2105

In [11]:
df[df.RouteName.str.len()<5]

,CRASHID,RouteName,BeginMilePost,RoadView,CrashSeverity,MANNERCOLLISIONID,ROADWAYJUNCTFEATUREID,NUMBERVEHICLESINVOLVED,FIRSTHARMFULEVENTID,ROADWAYTYPECD,ROUTEDIRECTION,EXITNUMBER,RAMPID,Latitude,Longitude,db,project,run,lat,lon,dayofyear,year,month,day,hour,dayofweek,incident
DATE,,,,,,,,,,,,,,,,,,,,,,,,,,,


### Make LABEL column to join with Safety Index Layer

The RouteName contains just the road names. If the crash occurred on a ramp or connector, RAMPID is a number and the ramp/connector label is in the project column. The LABEL column reads these from the RouteName or project column conditionally. In some cases, RAMPID is a number but project does not include the ramp label (which is supposed to be 11 chars long). Handle that as well.

In [12]:
df['LABEL'] = df.project.where(df.RAMPID.notnull() & (df.project.str.len()==11), df['RouteName'])

In [13]:
df[df.LABEL.str.len()==11].shape

(16108, 28)

In [14]:
df[df.LABEL.str.len()<11].head()

,CRASHID,RouteName,BeginMilePost,RoadView,CrashSeverity,MANNERCOLLISIONID,ROADWAYJUNCTFEATUREID,NUMBERVEHICLESINVOLVED,FIRSTHARMFULEVENTID,ROADWAYTYPECD,ROUTEDIRECTION,EXITNUMBER,RAMPID,Latitude,Longitude,db,project,run,lat,lon,dayofyear,year,month,day,hour,dayofweek,incident,LABEL
DATE,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-10-15 19:56:00,10401151,0009P,11.64,http://168.178.125.111/virtualnavigator/udot/V...,2,Front to Rear,No Special Feature/Junction,2,Other Motor Vehicle in Transport,Mainline,P,NaN,NaN,4119389.0,298229.0,2014,0009,P,37.199177,-113.273482,288,2010,10,15,19,4,1,0009P
2010-08-20 19:33:00,10394697,0009P,11.85,http://168.178.125.111/virtualnavigator/udot/V...,1,Sideswipe Same Direction,No Special Feature/Junction,2,Other Motor Vehicle in Transport,Mainline,P,NaN,NaN,4119704.0,298349.0,2014,0009,P,37.202040,-113.272216,232,2010,8,20,19,4,1,0009P
2010-03-27 21:30:00,10346333,0009P,14.00,http://168.178.125.111/virtualnavigator/udot/V...,1,Single Vehicle,No Special Feature/Junction,1,Delineator Post,Mainline,P,NaN,NaN,4122320.0,299671.0,2014,0009,P,37.225887,-113.258033,86,2010,3,27,21,5,1,0009P
2010-04-16 13:45:00,10352753,0009P,14.00,http://168.178.125.111/virtualnavigator/udot/V...,1,Single Vehicle,No Special Feature/Junction,1,Overturn/Rollover,Mainline,P,NaN,NaN,4122320.0,299671.0,2014,0009,P,37.225887,-113.258033,106,2010,4,16,13,4,1,0009P
2010-03-05 09:48:00,10357506,0009P,21.00,http://168.178.125.111/virtualnavigator/udot/V...,1,Single Vehicle,No Special Feature/Junction,1,Animal - Wild,Mainline,P,NaN,NaN,4118637.0,309403.0,2014,0009,P,37.194752,-113.147468,64,2010,3,5,9,4,1,0009P


### Safety Index layer

In [4]:
from arcgis.features import SpatialDataFrame, FeatureLayer
safetyindex_lyr = FeatureLayer('https://maps.udot.utah.gov/arcgis/rest/services/Safety_SafetyIndex/MapServer/0')
safetyindex_df = SpatialDataFrame.from_layer(safetyindex_lyr)

In [8]:
safetyindex_df.head()

,AVG_AADT,AvE_CRT,AvE_SEV_CR,BEG_MP,CRT,C_MI,DESCRIP,END_MP,EXP_CRT,EXP_SEV_CR,FC_NAME,LABEL,OBJECTID,PROTECTED,REGION,RouteNm,SAF_INDEX,SEV_CRT,SEV_C_MI,State_FA,TOT_C,TOT_SEV_C,SHAPE
0,16786.666667,0.416872,0.510455,88.353,0.312654,1.915709,SR 271 Paragonah,94.453,0.75,3.5,Rural Interstate,0015N,1,23 USC 409,4,I-15,3.5,1.786592,0.109469,State,35,2,"{'paths': [[[-12549375.5384, 4578057.411700003..."
1,86801.666667,0.628322,0.000000,7.697,0.703721,22.295806,SR 152 2000 East (Highland Drive),9.216,1.12,1.4,Urban Interstate,0215P,2,23 USC 409,2,I-215,2.5,0.000000,0.000000,State,101,0,"{'paths': [[[-12449309.0297, 4958382.946800001..."
2,19135.000000,1.097356,0.000000,0.000,3.006754,21.000000,Bangerter Highway via 10400 South,1.004,2.74,6.8,Urban Principal Arterial,0151P,3,23 USC 409,2,SR-151,3.0,0.000000,0.000000,State,63,0,"{'paths': [[[-12465232.5222, 4947938.317000002..."
3,2836.666667,1.745239,3.620154,7.072,4.677239,4.843305,Rt turns on 6800 South - SR 156,8.247,2.68,7.6,Urban Collector,0115P,4,23 USC 409,3,SR-115,7.0,27.513173,0.284900,State,17,1,"{'paths': [[[-12431824.1703, 4881649.121600002..."
4,13583.588402,1.355949,0.650438,1.343,3.715300,18.421053,SR 90 (200 S) via SR 13 (Main St) - Forest St ...,2.872,2.74,6.8,Urban Principal Arterial,0013P,5,23 USC 409,1,SR-13,6.0,4.422976,0.219298,State,84,1,"{'paths': [[[-12469545.3113, 5087281.274599999..."


In [6]:
safetyindex_df.shape

(3667, 23)

### Choosing which data points to use

In [17]:
crashes_ramps = df[df['LABEL'].str.len()==11]  # crashes on ramps

In [18]:
crashes_roads = df[df['LABEL'].str.len()!=11]  # crashes on roads

In [19]:
len(crashes_ramps.LABEL.unique())

1062

In [20]:
len(crashes_roads.LABEL.unique())

2099

In [21]:
len(safetyindex_df.LABEL.unique())   # 2099 - 904 roads will not be matched, hopefully they're minor roads

904

In [22]:
len(crashes_roads[crashes_roads['LABEL'].str.len()>5]['LABEL'].unique())

406

In [23]:
len(crashes_roads[crashes_roads['LABEL'].str.len()==5]['LABEL'].unique())

1693

In [24]:
crashes_roads[crashes_roads['LABEL'].str.len()<5]['LABEL']

Series([], Name: LABEL, dtype: object)

In [25]:
len(crashes_ramps), len(crashes_roads)

(16108, 414637)

In [26]:
crashes_roads[crashes_roads.LABEL.isin(safetyindex_df.LABEL.unique())].shape

(342376, 28)

In [27]:
len(crashes_ramps)

16108

In [28]:
342376/414637          # data points ignoring ramps, have all data

0.8257246700125652

In [29]:
(342376+16108)/414637  # data points including ramps, missing ramp specific data from safety index layer, can use road data

0.8645731085262531

In [30]:
372524/414637          # data points if using LRS layer, calculated in another notebook - missing safety index data

0.8984340519538777

We'll not be able to use approx 3% of the crash data if we use safety index layer instead of LRS layer. However, LRS layer is missing the data we want to use (AADT, safety rating, crashes per mile, etc). Proceeding with Safety Index layer.

In [31]:
crashes_roads_segments = pd.merge(crashes_roads, safetyindex_df, on='LABEL')

In [32]:
crashes_roads_segments.shape

(15003075, 50)

In [33]:
crashes_roads_segments = crashes_roads_segments[crashes_roads_segments.BeginMilePost.between(crashes_roads_segments.BEG_MP, crashes_roads_segments.END_MP)]

In [34]:
crashes_roads_segments.shape

(342748, 50)

In [35]:
crashes_roads_segments.head()

,CRASHID,RouteName,BeginMilePost,RoadView,CrashSeverity,MANNERCOLLISIONID,ROADWAYJUNCTFEATUREID,NUMBERVEHICLESINVOLVED,FIRSTHARMFULEVENTID,ROADWAYTYPECD,ROUTEDIRECTION,EXITNUMBER,RAMPID,Latitude,Longitude,db,project,run,lat,lon,dayofyear,year,month,day,hour,dayofweek,incident,LABEL,AVG_AADT,AvE_CRT,AvE_SEV_CR,BEG_MP,CRT,C_MI,DESCRIP,END_MP,EXP_CRT,EXP_SEV_CR,FC_NAME,OBJECTID,PROTECTED,REGION,RouteNm,SAF_INDEX,SEV_CRT,SEV_C_MI,State_FA,TOT_C,TOT_SEV_C,SHAPE
11,10401151,0009P,11.64,http://168.178.125.111/virtualnavigator/udot/V...,2,Front to Rear,No Special Feature/Junction,2,Other Motor Vehicle in Transport,Mainline,P,NaN,NaN,4119389.0,298229.0,2014,0009,P,37.199177,-113.273482,288,2010,10,15,19,4,1,0009P,11829.619048,0.287125,0.000000,10.883,0.786722,3.397028,860 N (Enchanted Way) via State St - 300 S La ...,12.458,2.74,6.8,Urban Principal Arterial,2045,23 USC 409,4,SR-9,2.0,0.000000,0.000000,State,16,0,"{'paths': [[[-12610092.850200001, 4465474.1281..."
23,10394697,0009P,11.85,http://168.178.125.111/virtualnavigator/udot/V...,1,Sideswipe Same Direction,No Special Feature/Junction,2,Other Motor Vehicle in Transport,Mainline,P,NaN,NaN,4119704.0,298349.0,2014,0009,P,37.202040,-113.272216,232,2010,8,20,19,4,1,0009P,11829.619048,0.287125,0.000000,10.883,0.786722,3.397028,860 N (Enchanted Way) via State St - 300 S La ...,12.458,2.74,6.8,Urban Principal Arterial,2045,23 USC 409,4,SR-9,2.0,0.000000,0.000000,State,16,0,"{'paths': [[[-12610092.850200001, 4465474.1281..."
33,10346333,0009P,14.00,http://168.178.125.111/virtualnavigator/udot/V...,1,Single Vehicle,No Special Feature/Junction,1,Delineator Post,Mainline,P,NaN,NaN,4122320.0,299671.0,2014,0009,P,37.225887,-113.258033,86,2010,3,27,21,5,1,0009P,3031.666667,0.463938,0.000000,13.282,0.626316,0.693132,100 East La Verkin - Mill St Virgin,18.580,1.35,5.1,Rural Principal Arterial,1631,23 USC 409,4,SR-9,1.0,0.000000,0.000000,State,11,0,"{'paths': [[[-12608433.0227, 4469426.418499999..."
45,10352753,0009P,14.00,http://168.178.125.111/virtualnavigator/udot/V...,1,Single Vehicle,No Special Feature/Junction,1,Overturn/Rollover,Mainline,P,NaN,NaN,4122320.0,299671.0,2014,0009,P,37.225887,-113.258033,106,2010,4,16,13,4,1,0009P,3031.666667,0.463938,0.000000,13.282,0.626316,0.693132,100 East La Verkin - Mill St Virgin,18.580,1.35,5.1,Rural Principal Arterial,1631,23 USC 409,4,SR-9,1.0,0.000000,0.000000,State,11,0,"{'paths': [[[-12608433.0227, 4469426.418499999..."
49,10357506,0009P,21.00,http://168.178.125.111/virtualnavigator/udot/V...,1,Single Vehicle,No Special Feature/Junction,1,Animal - Wild,Mainline,P,NaN,NaN,4118637.0,309403.0,2014,0009,P,37.194752,-113.147468,64,2010,3,5,9,4,1,0009P,1881.666667,1.242143,2.055015,18.580,1.676893,1.151908,Mill Street Virgin - Center St Rockville,27.847,1.35,5.1,Rural Principal Arterial,375,23 USC 409,4,SR-9,5.0,10.480579,0.071994,State,32,2,"{'paths': [[[-12600111.84, 4467519.9580999985]..."


In [36]:
crashes_roads_segments.to_pickle('crashes_roads_segments.pkl')

In [37]:
crashes_roads_segments.to_csv('crashes_roads_segments.csv')

In [9]:
safetyindex_df.to_csv('safetyindex_roads.csv')